In [1]:
import sys 
sys.path.append('../')
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from main import Pipeline as Manager


m = Manager('../')


Options db Connected: 2024-11-08 13:08:18.413377
Prices Connected: 2024-11-08 13:08:18.414479


In [ ]:
# Not Implemented Yet. 

# Check for earnings in the next 5 days 
# m.Earningsdb.update_earnings()


# earnings_df = m.Earningsdb.upcoming_earnings(n = 10)
# edf = m.Earningsdb.earnings_calendar(n = 10)
# earnings_df = earnings_df.rename(columns = {'stock':'symbol'})
# edf.symbol = edf.symbol.str.upper()

2,332 Companies report Earnings in the next 10 days, 13 are currently tracked.


In [2]:
# print(edf[edf.reportDate == edf.reportDate.min()].shape)
# edf[edf.reportDate == edf.reportDate.min()]

## Start with Daily Option Statistics

In [4]:
df = pd.read_sql('select * from spy', m.Optionsdb.vol_db)
df.atm_iv.iloc[-1], df.atm_iv.mean()

(0.009026130480743437, 0.012978798356115099)

In [5]:
daily_option_stats = pd.read_sql('select * from daily_option_stats', m.Optionsdb.stats_db).drop(columns = 'gatherdate').set_index(['stock'])
drop_cols = daily_option_stats.filter(like = 'pct').columns
daily_option_stats = daily_option_stats.drop(columns = drop_cols)

In [6]:
print(f'''
Total Volume: {daily_option_stats.total_vol.sum():,.0f}
Total OI: {daily_option_stats.total_oi.sum():,.0f}\t
Total Prem: ${daily_option_stats.total_prem.sum():,.0f}

Calls:
OI: {daily_option_stats.call_oi_chng.sum():,.0f}\tIV: {daily_option_stats.call_iv_chng.mean():,.2f}\tΔPremium: {daily_option_stats.call_prem_chng.sum():,.2f}

Puts:
OI: {daily_option_stats.put_oi_chng.sum():,.0f}\tIV: {daily_option_stats.put_iv_chng.mean():,.2f}\tΔPremium: {daily_option_stats.put_prem_chng.sum():,.2f}
''')


Total Volume: 9,545,327
Total OI: 61,010,561	
Total Prem: $1,730,082,060

Calls:
OI: 893,173	IV: 0.01	ΔPremium: 19,522,990.32

Puts:
OI: 1,281,106	IV: 0.02	ΔPremium: 36,048,415.37



In [7]:
# Highest Volume For Today
vol_cols = daily_option_stats.filter(like = 'vol').columns
top_volume = daily_option_stats[daily_option_stats.group.isin(['market','mag8']) == False][vol_cols]
top_volume['pcr'] = top_volume.put_vol / top_volume.call_vol
top_volume.sort_values(
    'total_vol',
    ascending = False
    ).head(10).style.format("{:,.0f}")


top_volume.sort_values('total_vol', ascending = False).head(10).style.format("{:,.0f}") 

,avg_vol,pcr_vol,avg_pcr_vol,call_vol,put_vol,total_vol,call_vol_chng,put_vol_chng,total_vol_chng,pcr
stock,,,,,,,,,,
smci,"381,859",1,1,"135,228","119,226","254,454","-136,312","-65,804","-202,116",1
djt,"228,705",1,1,"145,036","89,987","235,023","-87,186","-94,678","-181,864",1
tlt,"262,291",0,1,"145,083","49,882","194,965","-34,717","-32,200","-66,917",0
amd,"339,754",1,1,"124,451","66,517","190,968","-60,182","-23,990","-84,172",1
pltr,"266,835",1,1,"87,327","79,580","166,907","-126,623","-106,482","-233,105",1
wfc,"41,284",0,1,"133,627","10,855","144,482","107,013","-31,206","75,807",0
coin,"159,573",0,0,"88,502","43,061","131,563","-96,556","-34,625","-131,181",0
c,"78,097",0,0,"105,629","21,556","127,185","3,391","5,328","8,719",0
sofi,"200,983",0,0,"86,295","27,265","113,560","-18,568","-10,462","-29,030",0


In [8]:
# Highest OI Change For Today
oi_cols = daily_option_stats.filter(like = 'oi').columns
daily_option_stats[daily_option_stats.group.isin(['market','mag8']) == False][oi_cols].sort_values(
    'total_oi_chng', 
    ascending = False
    ).head(10).style.format("{:,.0f}")  

,pcr_oi,avg_pcr_oi,avg_oi,total_oi_std,call_oi,put_oi,total_oi,call_oi_chng,put_oi_chng,total_oi_chng
stock,,,,,,,,,,
tlt,1,1,"1,902,425","390,339","1,384,676","970,089","2,354,765","33,420","98,263","131,683"
dkng,0,1,"198,813","78,979","246,980","112,392","359,372","70,050","27,637","97,687"
sq,1,1,"138,599","34,369","111,439","118,729","230,168","23,227","65,851","89,078"
lqd,2,1,"428,272","101,425","156,979","306,004","462,983","-1,661","66,514","64,853"
xlf,1,1,"964,051","272,913","752,560","715,253","1,467,813","29,978","29,524","59,502"
abnb,1,1,"91,816","21,627","71,378","74,358","145,736","23,419","26,612","50,031"
afrm,1,1,"196,418","44,237","157,820","84,009","241,829","32,606","15,747","48,353"
intc,1,1,"919,620","152,249","603,057","404,368","1,007,425","22,606","24,818","47,424"
gld,1,1,"938,588","125,357","507,499","454,537","962,036","19,548","23,930","43,478"


### Whos pricing in the most? 


In [9]:
expected = pd.read_sql('select * from expected_moves where stk_price > 10.00 order by empct desc', m.Optionsdb.stats_db, parse_dates = ['expiry'])
expected.sort_values('empct', ascending = False).head(10)

,stock,expiry,stk_price,em,empct,twt
0,djt,2024-11-15,30.15,8.50,0.267828,$DJT ±8.50 (26.78%) 🔥
1,sono,2024-11-15,13.70,3.40,0.235766,$SONO ±3.40 (23.58%) 🔥
2,smci,2024-11-15,24.89,4.31,0.164504,$SMCI ±4.31 (16.45%) 🔥
3,jks,2024-11-15,23.41,4.05,0.164353,$JKS ±4.05 (16.44%) 🔥
4,clsk,2024-11-15,13.57,2.23,0.156116,$CLSK ±2.23 (15.61%) 🔥
5,spot,2024-11-15,400.83,55.95,0.132606,$SPOT ±55.95 (13.26%) 🔥
6,tan,2024-11-08,35.98,4.89,0.129113,$TAN ±4.89 (12.91%) 🔥
7,mara,2024-11-15,19.29,2.55,0.125583,$MARA ±2.55 (12.56%) 🔥
8,cava,2024-11-15,145.99,19.00,0.123639,$CAVA ±19.00 (12.36%) 🔥
9,icln,2024-11-08,12.34,1.54,0.118558,$ICLN ±1.54 (11.86%) 🔥


In [ ]:
#  Joining the Earnings and Expected move DF 


### Now join the earnings_df with the expected moves
# earn_df = earnings_df[(earnings_df.Date == earnings_df.Date.min()) & (earnings_df.Time == 'PM')].copy()
# earn_df = earn_df.rename(columns = {'symbol':'stock'})
# earn_df.stock = [x.lower() for x in earn_df.stock]
# earn_df.set_index('stock').merge(expected.set_index('stock'), how = 'left', left_index = True, right_index = True).sort_values('Date', ascending = True)

### Do the same with edf 

# pedf = edf[edf.reportDate == edf.reportDate.min()].copy()
# pedf = pedf.rename(columns = {'symbol':'stock', 'reportDate':'Date'}) 
# pedf.stock = [x.lower() for x in pedf.stock]
# pedf.set_index('stock').merge(expected.set_index('stock'), how = 'left', left_index = True, right_index = True).sort_values('Date', ascending = True)
# print(pedf.stock.to_list())


,Date,Hour,Time,estimated_eps,expiry,stk_price,em,empct,twt
stock,,,,,,,,,


### Now lets get some info on hot contracts 

In [12]:
# Contracts with the highest Dealer Interest (OI Change)
ddod = pd.read_sql('select * from amnt where oi_chg > 1000 ', m.Optionsdb.stats_db)
print(ddod.shape)
ddod = m.Optionsdb.pcdb(ddod)
ddod.sort_values('amnt', ascending = False).head(12)[
    ['gatherdate','stock', 'expiry','type', 'strike','stk_price', 'lastprice','volume', 'openinterest','amnt', ]
    ]

(100, 27)


,gatherdate,stock,expiry,type,strike,stk_price,lastprice,volume,openinterest,amnt
59,2024-11-08 11:01:33,LQD,2024-11-15,Put,109.0,109.88,0.20,1,54008,43621
71,2024-11-08 11:09:00,IWM,2024-11-15,Call,245.0,237.24,0.59,5630,59077,34962
37,2024-11-08 10:51:48,SLV,2025-03-31,Call,36.0,28.59,0.51,19,29134,26573
38,2024-11-08 10:51:48,SLV,2025-03-31,Call,41.0,28.59,0.28,13641,29942,13596
39,2024-11-08 10:51:48,SLV,2025-03-31,Call,31.0,28.59,1.48,19,42061,10664
0,2024-11-08 10:40:47,GDX,2025-01-17,Call,47.0,39.06,0.25,285,70977,9991
45,2024-11-08 10:55:23,QQQ,2024-12-31,Put,455.0,513.42,1.64,9,14135,8928
10,2024-11-08 10:43:22,XLF,2024-11-29,Call,50.0,49.31,0.35,2,72338,8496
57,2024-11-08 11:00:43,BITO,2024-11-15,Put,21.0,20.65,0.79,24,7882,7472
4,2024-11-08 10:41:14,AMZN,2024-11-15,Call,225.0,208.22,0.11,81,15095,7231


## Highest Percent Change Contracts 

In [13]:
high_pct = pd.read_sql('select * from pct_chg ', m.Optionsdb.stats_db)

high_pct = m.Optionsdb.pcdb(high_pct)
# high_pct.expiry = pd.to_datetime(high_pct.expiry)
# high_pct.gatherdate = pd.to_datetime(high_pct.gatherdate)
# high_pct['dte'] = (high_pct.expiry - high_pct.gatherdate).dt.days
high_pct.sort_values('pct_chg', ascending = False).head(12)[
    ['gatherdate','contractsymbol','stock', 'expiry','type', 'strike','stk_price','lastprice','volume', 'openinterest','pct_chg']
]


,gatherdate,contractsymbol,stock,expiry,type,strike,stk_price,lastprice,volume,openinterest,pct_chg
0,2024-11-08 10:55:23,QQQ241113P00496000,QQQ,2024-11-13,Put,496.0,513.42,0.28,36,130,2700.0
2,2024-11-08 11:07:11,CVS241108P00056000,CVS,2024-11-08,Put,56.0,56.86,0.24,360,866,2300.0
3,2024-11-08 11:08:47,ROKU241129P00061000,ROKU,2024-11-29,Put,61.0,71.58,0.16,15,79,1500.0
4,2024-11-08 11:13:07,SONY241108C00020200,SONY,2024-11-08,Call,20.2,20.04,0.30,196,15,1400.0
1,2024-11-08 11:07:01,CSCO241213C00066000,CSCO,2024-12-13,Call,66.0,58.03,0.61,10,14,1120.0


### Cheap Compared to Recent IV 

In [14]:
iv_dif = pd.read_sql('select * from iv_diff where impliedvolatility > 0.01 and lastprice > 0.10 and lastprice < 2.00 order by iv_diff desc ', m.Optionsdb.stats_db)
iv_dif = m.Optionsdb.pcdb(iv_dif)
iv_dif.sort_values('iv_diff', ascending = False).head(12)[
    ['gatherdate','contractsymbol','stock', 'expiry','type', 'strike','stk_price','lastprice','lastprice_avg_30d','impliedvolatility', 'iv_avg_30d','iv_diff']
]

,gatherdate,contractsymbol,stock,expiry,type,strike,stk_price,lastprice,lastprice_avg_30d,impliedvolatility,iv_avg_30d,iv_diff
0,2024-11-08 11:17:52,DJT241108P00029500,DJT,2024-11-08,Put,29.5,30.15,0.75,4.820000,1.593752,3.659931,2.066179
1,2024-11-08 11:17:52,DJT241108P00028500,DJT,2024-11-08,Put,28.5,30.15,0.36,4.425333,1.617189,3.673571,2.056381
2,2024-11-08 11:17:52,DJT241108P00027500,DJT,2024-11-08,Put,27.5,30.15,0.23,3.936667,1.742189,3.692060,1.949872
3,2024-11-08 11:17:52,DJT241108C00033500,DJT,2024-11-08,Call,33.5,30.15,0.28,9.170000,2.000005,3.859476,1.859471
4,2024-11-08 11:17:52,DJT241108P00029000,DJT,2024-11-08,Put,29.0,30.15,0.50,5.490000,1.660158,3.479641,1.819483
5,2024-11-08 11:03:14,AFRM241108P00044500,AFRM,2024-11-08,Put,44.5,44.06,0.48,3.288000,0.132821,1.485746,1.352925
6,2024-11-08 11:17:09,SMCI241108C00024500,SMCI,2024-11-08,Call,24.5,24.89,0.49,2.186000,0.527348,1.680472,1.153123
7,2024-11-08 11:17:09,SMCI241108C00025000,SMCI,2024-11-08,Call,25.0,24.89,0.24,1.910000,0.589848,1.694143,1.104296
8,2024-11-08 11:17:09,SMCI241108P00024500,SMCI,2024-11-08,Put,24.5,24.89,0.39,1.760000,1.023442,2.045901,1.022459
9,2024-11-08 11:17:09,SMCI241108P00023500,SMCI,2024-11-08,Put,23.5,24.89,0.13,1.370000,1.125004,2.092971,0.967967


### Highest Inflows

In [15]:
# This needs to be fixed, becuase as of rn; it only updates when the previous tweets were generated. 
inflow = pd.read_sql('select * from voi where voi > 1 and lastprice < 1.00 and volume > 1000 order by voi desc ', m.Optionsdb.stats_db)
inflow = m.Optionsdb.pcdb(inflow)
inflow['dte'] = pd.to_datetime(inflow.expiry) - pd.to_datetime(inflow.gatherdate)
inflow['dte'] = inflow.dte.dt.days
inflow = inflow[inflow.dte > 10]
inflow.sort_values('volume', ascending = False).head(12)[
    ['gatherdate','stock', 'expiry','type', 'strike','stk_price', 'lastprice','lastprice_avg_5d','volume', 'openinterest','voi']
]

,gatherdate,stock,expiry,type,strike,stk_price,lastprice,lastprice_avg_5d,volume,openinterest,voi
4,2024-11-08 10:44:51,XLI,2024-11-29,Put,130.5,142.03,0.19,0.740000,30002,1072,27
0,2024-11-08 10:44:51,XLI,2024-11-29,Call,145.0,142.03,0.48,0.306667,22513,147,153
32,2024-11-08 11:15:31,MPW,2024-11-29,Put,4.5,4.82,0.22,0.315000,20907,4468,4
6,2024-11-08 11:00:01,XLB,2025-01-17,Put,86.0,94.79,0.53,0.805000,6000,241,24
29,2024-11-08 11:11:20,ABNB,2024-12-20,Put,120.0,135.64,0.95,1.831667,5277,1019,5
28,2024-11-08 11:10:28,C,2024-11-22,Call,73.0,69.03,0.21,0.221667,5045,905,5
52,2024-11-08 11:17:09,SMCI,2024-12-20,Put,15.0,24.89,0.95,1.356000,3968,1957,2
23,2024-11-08 11:11:56,MARA,2024-12-06,Put,17.0,19.29,0.91,1.780000,3027,388,7
21,2024-11-08 11:02:13,TLT,2024-12-13,Put,89.0,92.38,0.45,0.990000,2177,300,7
11,2024-11-08 11:01:59,JETS,2024-11-22,Put,24.0,24.47,0.37,0.732000,2052,166,12


In [16]:
# Other tables in the db, (I think we covered them all)
pd.read_sql('select * from sqlite_master', m.Optionsdb.stats_db)

,type,name,tbl_name,rootpage,sql
0,table,daily_option_stats,daily_option_stats,217,"CREATE TABLE ""daily_option_stats"" (\n""stock"" T..."
1,table,expected_moves,expected_moves,2,"CREATE TABLE ""expected_moves"" (\n""stock"" TEXT,..."
2,table,exp_ext,exp_ext,221,"CREATE TABLE ""exp_ext"" (\n""stock"" TEXT,\n ""ex..."
3,table,pct_chg,pct_chg,6,"CREATE TABLE ""pct_chg"" (\n""gatherdate"" TEXT,\n..."
4,table,volume,volume,9,"CREATE TABLE ""volume"" (\n""gatherdate"" TEXT,\n ..."
5,table,voi,voi,4,"CREATE TABLE ""voi"" (\n""gatherdate"" TEXT,\n ""c..."
6,table,iv_diff,iv_diff,7,"CREATE TABLE ""iv_diff"" (\n""gatherdate"" TEXT,\n..."
7,table,oi_chg,oi_chg,17,"CREATE TABLE ""oi_chg"" (\n""gatherdate"" TEXT,\n ..."
8,table,amnt,amnt,5,"CREATE TABLE ""amnt"" (\n""gatherdate"" TEXT,\n ""..."


In [17]:
pd.read_sql('select * from xom', m.Optionsdb.change_db)

,gatherdate,contractsymbol,stk_price,lastprice,ask,bid,change,percentchange,volume,openinterest,...,lastprice_avg_5d,pct_chg,iv_chg,iv_avg_5d,iv_avg_30d,iv_avg_all,vol_chg,oi_chg,flag,amnt
0,2024-05-06 13:54:40,PXD240510C00240000,117.35,31.00,0.00,0.00,0.0,0.0,1,1,...,31.00,NaN,NaN,0.500005,0.500005,0.500005,NaN,NaN,0,0
1,2024-05-07 13:02:47,PXD240510C00240000,117.01,31.00,0.00,0.00,0.0,0.0,1,1,...,31.00,0.0,0.000000,0.500005,0.500005,0.500005,0.0,0.0,0,0
2,2024-05-08 14:41:01,PXD240510C00240000,115.88,31.00,0.00,0.00,0.0,0.0,1,1,...,31.00,0.0,0.000000,0.500005,0.500005,0.500005,0.0,0.0,0,0
3,2024-05-09 15:28:45,PXD240510C00240000,118.35,31.00,0.00,0.00,0.0,0.0,1,1,...,31.00,0.0,0.000000,0.500005,0.500005,0.500005,0.0,0.0,0,0
4,2024-05-10 13:03:09,PXD240510C00240000,117.78,31.00,0.00,0.00,0.0,0.0,1,1,...,31.00,0.0,0.000000,0.500005,0.500005,0.500005,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248506,2024-10-29 15:52:55,XOM270115P00185000,117.16,65.47,69.60,66.05,0.0,0.0,0,0,...,65.47,0.0,-0.029022,0.235466,0.235466,0.235466,0.0,0.0,0,0
248507,2024-10-30 15:05:47,XOM270115P00185000,116.99,65.47,69.80,66.55,0.0,0.0,0,0,...,65.47,0.0,0.001587,0.231158,0.231158,0.231158,0.0,0.0,0,0
248508,2024-10-31 14:59:24,XOM270115P00185000,117.07,65.47,70.80,66.45,0.0,0.0,0,0,...,65.47,0.0,0.028991,0.236252,0.236252,0.236252,0.0,0.0,0,0
248509,2024-11-01 14:52:53,XOM270115P00185000,115.98,65.47,71.95,69.10,0.0,0.0,0,0,...,65.47,0.0,0.005676,0.240443,0.240443,0.240443,0.0,0.0,0,0


- ideas for more stats to add:

1. unusual spreads (ask - bid) 
2. 